# AFAP Orchestrator Validation Notebook
## Profile-by-Profile Simulation & Contract Assurance

### Purpose
This notebook validates the AFAP orchestrator across all supported
Analysis Interpretation Frameworks (AIFs).

Each profile is treated as an **independent client case**, ensuring:
- Correct engine selection
- Stable output contracts
- Profile-aware AI interpretation
- Reproducible, auditable execution

This notebook is intentionally structured to support:
- Engineering validation
- Audit defensibility
- Investor / stakeholder review


In [1]:
# ---------------------------------------------------------------
# Environment setup
# ---------------------------------------------------------------

import sys
import os
import pandas as pd
from pprint import pprint

# Ensure project root is on PYTHONPATH
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

print("Project root resolved to:", PROJECT_ROOT)


Project root resolved to: c:\Users\ADMIN\Documents\My Documents\MyDataAnalysis\Financial statement analysis\financial-analysis-pipeline


### Step 1: Import AFAP Orchestrator & Frozen Contracts

This cell imports:
- The AFAP orchestrator
- The frozen output contract
- All supported analysis profiles

These are treated as **non-negotiable system interfaces**.


In [2]:
from orchestrator.orchestrator import (
    afap_run,
    ANALYSIS_PROFILES,
    AFAP_OUTPUT_KEYS
)

pprint(ANALYSIS_PROFILES)


{'full_diagnostic': {'engines': ['ratio',
                                 'trend',
                                 'cash_flow',
                                 'anomaly',
                                 'solvency',
                                 'composite_risk'],
                     'metrics_scope': 'all'},
 'going_concern_screen': {'engines': ['ratio',
                                      'trend',
                                      'solvency',
                                      'composite_risk'],
                          'metrics_scope': 'critical_only'},
 'liquidity_focus': {'engines': ['ratio',
                                 'trend',
                                 'cash_flow',
                                 'composite_risk'],
                     'metrics_scope': ['current_ratio',
                                       'quick_ratio',
                                       'cash_ratio']},
 'performance_focus': {'engines': ['ratio', 'trend'],
                    

### Step 2: Load Financial Statements

This dataset simulates a **single recurring client**
across multiple years, allowing trend and risk analysis.


In [3]:
financials_df = pd.read_csv("../data/cleaned/Kenya_Airways.csv")
financials_df.head()


,Company,Year,FS Category,FS Subcategory,Amount
0,Kenya Airways,2021,Assets,Current Assets,"25,685"
1,Kenya Airways,2021,Assets,Non-Current Assets,"129,870"
2,Kenya Airways,2021,Assets,Inventory,"2,152"
3,Kenya Airways,2021,Liabilities,Current Liabilities,"80,965"
4,Kenya Airways,2021,Liabilities,Non-Current Liabilities,"157,927"


### Step 3: Define Validation Functions

These validators ensure:
- Output contract integrity
- AI interpretation structural consistency

They deliberately avoid business logic
and only validate **system correctness**.


In [4]:
def validate_output_contract(outputs: dict):
    """
    Ensures all required AFAP output keys are present.
    """
    missing = set(AFAP_OUTPUT_KEYS) - set(outputs.keys())
    assert not missing, f"Missing output keys: {missing}"


def validate_ai_interpretation(ai_output):
    """
    Validates AI interpretation structure (schema-level).
    """
    assert isinstance(ai_output, list)
    assert len(ai_output) > 0

    for rec in ai_output:
        assert "Company" in rec
        assert "Year" in rec
        assert "interpretation" in rec


### Step 4: Define Independent Client Analysis Cases

Each profile below represents a **realistic engagement scenario**:
- Targeted diagnostics
- Focused decision support
- Reduced computational scope

Each will be run **independently**.


In [5]:
CLIENT_CASES = [
    "full_diagnostic",
    "solvency_focus",
    "liquidity_focus",
    "performance_focus",
    "risk_scan",
    "going_concern_screen"
]


## Step 5: Execute a Single AFAP Profile (Case Simulation)

This cell is the **canonical execution pattern**.
It will be reused for every profile.

We:
1. Run the orchestrator
2. Validate the output contract
3. Validate AI interpretation
4. Inspect executed engines
5. Review sample outputs


In [6]:
# Select profile to simulate
profile_name = "solvency_focus"

print(f"\n--- Running AFAP Case: {profile_name} ---")

outputs = afap_run(
    financials_df=financials_df,
    analysis_profile=profile_name,
    use_mock_ai=False  # toggle True for fast dry-runs
)

# --- Contract validation ---
validate_output_contract(outputs)
print("✅ Output contract validated")

# --- AI validation ---
validate_ai_interpretation(outputs["ai_interpretation"])
print("✅ AI interpretation structure validated")

# --- Inspect executed engines ---
engines_ran = [
    k for k in AFAP_OUTPUT_KEYS
    if k in outputs and outputs[k]
]

print("Engines executed:", engines_ran)



--- Running AFAP Case: solvency_focus ---
✅ ratio_engine output validated successfully.
✅ solvency_engine output validated successfully.
✅ Output contract validated
✅ AI interpretation structure validated
Engines executed: ['profile_used', 'ratios', 'solvency', 'ai_interpretation']


### Step 6: Inspect AI Interpretation Output

This validates **semantic usability**, not correctness.


In [7]:
sample_interp = outputs["ai_interpretation"][0]

print("Company:", sample_interp["Company"])
print("Year:", sample_interp["Year"])
print("\n--- AI Interpretation ---\n")
print(sample_interp["interpretation"])


Company: Kenya Airways
Year: 2021

--- AI Interpretation ---

summary
Kenya Airways' 2021 financial ratios indicate material solvency and liquidity concerns. Short-term liquidity is weak (current_ratio 0.317, quick_ratio 0.291). Profitability measures are negative at the operating and net levels (operating_margin -9.69%, net_margin -22.84%), and return on assets is negative (roa -10.31%). Leverage indicators are atypical: debt_equity is -2.867 (negative), and interest_coverage is -0.725. Asset turnover is 0.451. Return on equity is reported positive (roe 19.25%), which is inconsistent with negative equity implied by the negative debt_equity ratio and should be subject to immediate verification.

key_risks
- Liquidity risk: current_ratio (0.317) and quick_ratio (0.291) are well below conservative thresholds (typically >=1.0), indicating insufficient short-term liquid resources to meet current obligations.
- Solvency / balance-sheet risk: debt_equity is negative (-2.867), which typically

### Step 7: Inspect Numerical Engine Outputs

This confirms:
- Ratios exist
- Engines populated data
- Schema consistency


In [8]:
if outputs.get("ratios"):
    display(pd.DataFrame(outputs["ratios"]).head())

if outputs.get("solvency"):
    display(pd.DataFrame(outputs["solvency"]).head())


,engine,Company,Year,metrics,flags,severity,explanation
0,ratio_engine,Kenya Airways,2021,"{'current_ratio': 0.3172358426480578, 'quick_r...",{},stable,Canonical financial ratios
1,ratio_engine,Kenya Airways,2022,"{'current_ratio': 0.3553302522298208, 'quick_r...",{},stable,Canonical financial ratios
2,ratio_engine,Kenya Airways,2023,"{'current_ratio': 0.41900024909402095, 'quick_...",{},stable,Canonical financial ratios
3,ratio_engine,Kenya Airways,2024,"{'current_ratio': 0.34834771979471035, 'quick_...",{},stable,Canonical financial ratios


,engine,Company,Year,metrics,flags,severity,explanation
0,solvency_engine,Kenya Airways,2021,"{'debt_equity': -2.866577870573695, 'interest_...","{'high_leverage': False, 'weak_coverage': True}",watch,Capital structure shows solvency risk.
1,solvency_engine,Kenya Airways,2022,"{'debt_equity': -2.5626977518734386, 'interest...","{'high_leverage': False, 'weak_coverage': True}",watch,Capital structure shows solvency risk.
2,solvency_engine,Kenya Airways,2023,"{'debt_equity': -2.3023474790494194, 'interest...","{'high_leverage': False, 'weak_coverage': True}",watch,Capital structure shows solvency risk.
3,solvency_engine,Kenya Airways,2024,"{'debt_equity': -2.5146087559513237, 'interest...","{'high_leverage': False, 'weak_coverage': True}",watch,Capital structure shows solvency risk.


## Step 8: Full System Validation (All Profiles)

This cell simulates **multiple client engagements**,
ensuring no profile breaks the orchestrator.


In [9]:
results_by_profile = {}

for profile_name in CLIENT_CASES:
    print(f"\n--- Executing profile: {profile_name} ---")

    outputs = afap_run(
        financials_df=financials_df,
        analysis_profile=profile_name,
        use_mock_ai=False
    )

    validate_output_contract(outputs)
    validate_ai_interpretation(outputs["ai_interpretation"])

    engines_ran = [
        k for k in AFAP_OUTPUT_KEYS
        if k in outputs and outputs[k]
    ]

    print("Engines executed:", engines_ran)

    results_by_profile[profile_name] = outputs

print("\n✅ All AFAP profiles validated successfully")



--- Executing profile: full_diagnostic ---
✅ ratio_engine output validated successfully.
✅ trend_engine output validated successfully.
✅ cash_flow_engine output validated successfully.
✅ anomaly_efficiency_engine output validated successfully.
✅ solvency_engine output validated successfully.
Engines executed: ['profile_used', 'ratios', 'trend', 'cash_flow', 'anomaly', 'solvency', 'composite_risk', 'ai_interpretation']

--- Executing profile: solvency_focus ---
✅ ratio_engine output validated successfully.
✅ solvency_engine output validated successfully.
Engines executed: ['profile_used', 'ratios', 'solvency', 'ai_interpretation']

--- Executing profile: liquidity_focus ---
✅ ratio_engine output validated successfully.
✅ trend_engine output validated successfully.
✅ cash_flow_engine output validated successfully.
Engines executed: ['profile_used', 'ratios', 'trend', 'cash_flow', 'composite_risk', 'ai_interpretation']

--- Executing profile: performance_focus ---
✅ ratio_engine output v

### Step 9: Compare AI Interpretations Across Profiles

This demonstrates the **profile-aware reasoning differences**.


In [10]:
for profile, outputs in results_by_profile.items():
    interp_list = outputs["ai_interpretation"]
    print(f"\n=== {profile.upper()} ===")
    for record in interp_list:
        print(f"{record['Company']} ({record['Year']}):")
        print(record["interpretation"][:20000], "...\n")



=== FULL_DIAGNOSTIC ===
Kenya Airways (2021):
- summary
  - This diagnostic assesses Kenya Airways (2021) using the provided financial ratios. Key liquidity metrics are materially below conservative thresholds. Profitability and coverage measures show operating and net losses and inadequate ability to service interest. Leverage indicators imply negative shareholders’ equity; this materially affects interpretability of return-on-equity. The firm’s asset utilisation is modest. Overall, the ratios reflect material liquidity, solvency and profitability risks for 2021.

- key_risks
  - Liquidity shortfall: current_ratio = 0.317 (conservative threshold: >=1.0). quick_ratio = 0.291 (conservative threshold: >=1.0). Both ratios exceed conservative risk thresholds, indicating potential near-term liquidity stress and limited ability to meet short-term obligations without additional funding or asset sales.
  - Operating and net losses: operating_margin = -0.0969 (threshold: >=0). net_margin = -0.

In [11]:
import pandas as pd

# List to collect all interpretations
all_records = []

for profile, outputs in results_by_profile.items():
    for record in outputs["ai_interpretation"]:
        all_records.append({
            "Profile": profile,
            "Company": record["Company"],
            "Year": record["Year"],
            "Interpretation": record["interpretation"]
        })

# Convert to DataFrame
df_interp = pd.DataFrame(all_records)

# Save to CSV
df_interp.to_csv("afap_ai_interpretations.csv", index=False, encoding="utf-8")

print("✅ CSV saved as 'afap_ai_interpretations.csv'")


✅ CSV saved as 'afap_ai_interpretations.csv'
